# Оптимизация выполнения кода, векторизация, Numba

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* IPython Cookbook, Second Edition (2018), глава 4
* https://ipython-books.github.io/43-profiling-your-code-line-by-line-with-line_profiler/
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html
* https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

## Задачи для совместного разбора

In [7]:
import numpy as np
from numba import njit

In [8]:
import numba
numba.__version__

'0.55.1'

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [9]:
A = np.random.randint(0, 1000, size=(1_000_000))
A

def f1(A):
    acc, cnt = 0, 0
    for i in range(len(A)):
        bi = A[i] + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [10]:
%%time
f1(A)

Wall time: 502 ms


599.14043

In [11]:
%time f1(A)

Wall time: 472 ms


599.14043

In [12]:
def f2(A):
    acc = 0
    for i in range(len(A)):
        acc += A[i]
    return acc / len(A) + 100

In [13]:
%time f2(A)

Wall time: 208 ms


599.1404299999999

In [14]:
%time A.mean() + 100

Wall time: 4 ms


599.1404299999999

In [18]:
@njit
def f3(A):
    acc = 0
    for i in range(len(A)):
        acc += A[i]
    return acc / len(A) + 100

In [24]:
%timeit f3(A)

86.1 µs ± 5.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Примените функцию к каждому элементу массива.

In [13]:
def g(x):
    return sum(range(x))

In [14]:
%%time
np.array([g(a) for a in A])

Wall time: 25.6 s


array([116886,  62128, 170236, ..., 220116,  10585,   1596])

In [15]:
g_v = np.vectorize(g)
g_v(A)

array([116886,  62128, 170236, ..., 220116,  10585,   1596])

In [16]:
%%time
g_v(A)

Wall time: 22.3 s


array([116886,  62128, 170236, ..., 220116,  10585,   1596])

In [17]:
A.dtype

dtype('int32')

In [18]:
g_v2 = numba.vectorize(["int32(int32)"])(g) 

In [19]:
%%time
g_v2(A)

Wall time: 129 ms


array([116886,  62128, 170236, ..., 220116,  10585,   1596])

3. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [20]:
%load_ext line_profiler

In [21]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,1.352568,2.154098,1.029655,-1.294537,v
1,1.588012,0.268194,-0.579146,1.246078,q


In [22]:
def h1(df):
    mask = []
    for _, row in df.iterrows():
        mask.append(row["key"] in {"a", "b", "c", "d", "e"})
    return df[mask]

In [23]:
%%time
h1(df.head(50000))

Wall time: 4.11 s


,col0,col1,col2,col3,key
7,1.219213,-1.233336,-1.450651,-0.970965,b
9,-0.416004,-1.032928,-1.262088,-2.955540,e
12,-0.800565,1.400754,-0.858814,-0.113306,c
13,-0.933419,2.535895,0.745165,-0.428211,d
14,0.537601,0.317470,1.280936,1.396974,a
...,...,...,...,...,...
49976,0.175344,-0.279796,0.653248,0.367055,e
49977,0.562929,0.556024,0.170626,-0.988362,c
49983,0.716599,-0.039500,0.998008,-0.298128,e
49989,-0.036925,-0.817683,0.004950,1.857417,c


In [24]:
%%time
df[df["key"].isin({"a", "b", "c", "d", "e"})]

Wall time: 312 ms


,col0,col1,col2,col3,key
7,1.219213,-1.233336,-1.450651,-0.970965,b
9,-0.416004,-1.032928,-1.262088,-2.955540,e
12,-0.800565,1.400754,-0.858814,-0.113306,c
13,-0.933419,2.535895,0.745165,-0.428211,d
14,0.537601,0.317470,1.280936,1.396974,a
...,...,...,...,...,...
1999985,0.967548,-0.842871,-1.033069,0.059576,b
1999987,-0.597568,0.184619,-1.765527,-2.135385,b
1999990,0.635071,-0.134844,-0.671425,0.174149,d
1999995,-0.546703,-0.512708,0.321765,0.622675,a


In [25]:
%%time
df[df["key"].str.contains(r"[abcde]")]

Wall time: 2.54 s


,col0,col1,col2,col3,key
7,1.219213,-1.233336,-1.450651,-0.970965,b
9,-0.416004,-1.032928,-1.262088,-2.955540,e
12,-0.800565,1.400754,-0.858814,-0.113306,c
13,-0.933419,2.535895,0.745165,-0.428211,d
14,0.537601,0.317470,1.280936,1.396974,a
...,...,...,...,...,...
1999985,0.967548,-0.842871,-1.033069,0.059576,b
1999987,-0.597568,0.184619,-1.765527,-2.135385,b
1999990,0.635071,-0.134844,-0.671425,0.174149,d
1999995,-0.546703,-0.512708,0.321765,0.622675,a


In [26]:
%lprun -f h1 h1(df.head(50000))

## Лабораторная работа 3

In [27]:
import numpy as np
import pandas as pd
from numba import jit, njit
import numba
from typing import Union

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

In [28]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv', index_col = 0)

In [29]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'], format = '%Y-%m-%d')
recipes = recipes.drop(index= recipes[recipes['submitted'].dt.year != 2010].index)
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0
...,...,...,...,...,...,...,...,...
29897,zoe s chicken tarragon,441211,40,76559,2010-11-04,12.0,from a good housekeeping at my hair salon. ha...,9.0
29907,zucchini and noodle slice,412518,60,423555,2010-02-10,21.0,"a yummy, tasty slice packed with vegies and ri...",13.0
29915,zucchini bread bread machine,409757,220,539686,2010-01-22,7.0,"originally from a packet of red star yeast, th...",NaN
29926,zucchini chip cupcakes,406686,35,628076,2010-01-04,9.0,this is a great tasting recipe to use up zucch...,14.0


1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [30]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    sum_len = 0
    k = 0
    for _, row in df.iterrows():
        full_desc = row['name'] + ' ' + row['description']
        sum_len += len(full_desc)
        k +=1
    return sum_len/k

In [31]:
get_mean_len_A(recipes)

265.501300390117

1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [32]:
def get_mean_len_B(df: pd.DataFrame) -> float:
    return df.apply(lambda x: x['name'] + ' ' + x['description'], axis = 1).apply(len).mean()
    

In [33]:
get_mean_len_B(recipes)

265.501300390117

1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи векторизированных операций получаете среднюю длину полных описаний. 

In [34]:
def get_mean_len_C(df: pd.DataFrame) -> float:
    return (df['name'] + ' ' + df['description']).apply(len).mean()

In [35]:
get_mean_len_C(recipes)

265.501300390117

1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [36]:
%%time 
get_mean_len_A(recipes)

Wall time: 127 ms


265.501300390117

In [37]:
%%time 
get_mean_len_B(recipes)

Wall time: 47.8 ms


265.501300390117

In [38]:
%%time 
get_mean_len_C(recipes)

Wall time: 8 ms


265.501300390117

In [39]:
%timeit get_mean_len_A(recipes)

152 ms ± 9.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
%timeit get_mean_len_B(recipes)

54.1 ms ± 3.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
%timeit get_mean_len_C(recipes)

3.34 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [42]:
import re


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [43]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 106.667 s
File: <ipython-input-57-21baa068d50d>
Function: get_word_reviews_count at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def get_word_reviews_count(df):
     5         1         30.0     30.0      0.0      word_reviews = {}
     6    126680  240709209.0   1900.1     22.6      for _, row in df.dropna(subset=["review"]).iterrows():
     7    126679   73800625.0    582.6      6.9          recipe_id, review = row["recipe_id"], row["review"]
     8    126679   39023248.0    308.0      3.7          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
     9   6918689   52345249.0      7.6      4.9          for word in words:
    10   6792010   79648816.0     11.7      7.5              if word not in word_reviews:
    11     93059    1279798.0     13.8      0.1                  word_reviews[word] = []
    12   6792010   86449639.0     12.7      8.1              word_reviews[word].append(recipe_id)
    13                                           
    14         1         35.0     35.0      0.0      word_reviews_count = {}
    15    126680  235264392.0   1857.2     22.1      for _, row in df.dropna(subset=["review"]).iterrows():
    16    126679   43622958.0    344.4      4.1          review = row["review"]
    17    126679   38070285.0    300.5      3.6          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
    18   6918689   55124245.0      8.0      5.2          for word in words:
    19   6792010  121329931.0     17.9     11.4              word_reviews_count[word] = len(word_reviews[word])
    20         1         28.0     28.0      0.0      return word_reviews_count

Больше всего времени в программе занимает итерация по строкам (iterrows()) и по словам (для создания словаря). Также неоптимально присваивание переменных recipe_id, review = row["recipe_id"], row["review"], одна из которых вообще в дальнейшем не используется. 

2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [44]:
%timeit get_word_reviews_count(reviews)

41.4 s ± 1.18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
get_word_reviews_count(reviews)

{'Last': 97,
 'week': 1509,
 'whole': 5782,
 'sides': 458,
 'of': 109239,
 'frozen': 2943,
 'salmon': 1032,
 'fillet': 92,
 'was': 89617,
 'on': 36121,
 'sale': 262,
 'in': 62796,
 'my': 44644,
 'local': 572,
 'supermarket': 95,
 'so': 46848,
 'I': 288243,
 'bought': 1514,
 'tons': 163,
 'okay': 600,
 'only': 14209,
 '': 330966,
 'but': 43035,
 'total': 512,
 'weight': 216,
 'over': 9801,
 'pounds': 291,
 'This': 39555,
 'recipe': 71751,
 'is': 56519,
 'perfect': 7807,
 'for': 123162,
 'even': 8074,
 'though': 4942,
 'it': 133219,
 'calls': 525,
 'steaks': 511,
 'cut': 6848,
 'up': 16178,
 'the': 266530,
 'into': 7085,
 'individual': 314,
 'portions': 214,
 'and': 219215,
 'followed': 4880,
 'instructions': 1007,
 'exactly': 4625,
 'Im': 8330,
 'one': 17615,
 'those': 2419,
 'food': 3498,
 'combining': 78,
 'diets': 45,
 'left': 5064,
 'out': 25697,
 'white': 3690,
 'wine': 1760,
 'added': 22099,
 'just': 25271,
 'a': 166467,
 'dash': 547,
 'vinegar': 1946,
 'instead': 11644,
 'little'

In [47]:
def get_word_reviews_count_new(df):
    return df["review"].dropna().apply(lambda x: re.sub("[^A-Za-z\s]", "", x).split(" ")).explode().value_counts(sort=False).to_dict()

In [48]:
%timeit get_word_reviews_count_new(reviews)

6.08 s ± 199 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
get_word_reviews_count_new(reviews)

{'Last': 97,
 'week': 1509,
 'whole': 5782,
 'sides': 458,
 'of': 109239,
 'frozen': 2943,
 'salmon': 1032,
 'fillet': 92,
 'was': 89617,
 'on': 36121,
 'sale': 262,
 'in': 62796,
 'my': 44644,
 'local': 572,
 'supermarket': 95,
 'so': 46848,
 'I': 288243,
 'bought': 1514,
 'tons': 163,
 'okay': 600,
 'only': 14209,
 '': 330966,
 'but': 43035,
 'total': 512,
 'weight': 216,
 'over': 9801,
 'pounds': 291,
 'This': 39555,
 'recipe': 71751,
 'is': 56519,
 'perfect': 7807,
 'for': 123162,
 'even': 8074,
 'though': 4942,
 'it': 133219,
 'calls': 525,
 'steaks': 511,
 'cut': 6848,
 'up': 16178,
 'the': 266530,
 'into': 7085,
 'individual': 314,
 'portions': 214,
 'and': 219215,
 'followed': 4880,
 'instructions': 1007,
 'exactly': 4625,
 'Im': 8330,
 'one': 17615,
 'those': 2419,
 'food': 3498,
 'combining': 78,
 'diets': 45,
 'left': 5064,
 'out': 25697,
 'white': 3690,
 'wine': 1760,
 'added': 22099,
 'just': 25271,
 'a': 166467,
 'dash': 547,
 'vinegar': 1946,
 'instead': 11644,
 'little'

## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.



In [8]:
import json

with open(
    "rating_predictions.json",
    "r",
    encoding="utf-8"
) as fp:
    rating_predictions = json.load(fp)

In [9]:
A_list = []
F_list = []

for rating in rating_predictions:
    A_list.append(rating['rating'])
    F_list.append(rating['prediction'])

In [10]:
def mape_lists(A, F):
    mape = 0
    for r in range(len(A_list)):
        mape += abs((A[r]-F[r])/A[r])
    return 100*mape/len(A_list)

In [11]:
%%time
mape_lists(A_list, F_list)

Wall time: 88 ms


13.325265503992638

3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [12]:
A_array = np.array(A_list)
F_array = np.array(F_list)

In [18]:
def mape_numpy(A, F):
    return 100*np.sum(np.abs((A-F)/A))/len(A)

In [19]:
%%time
mape_numpy(A_array, F_array)

Wall time: 8 ms


13.32526550399145

3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_numpy` и `F_numpy`.

In [20]:
A_typed = numba.typed.List(A_list)
F_typed = numba.typed.List(F_list)

In [28]:
@njit
def mape_numba(A, F):
    mape = 0
    for r in range(len(A)):
        mape += np.abs((A[r]-F[r])/A[r])
    return 100*mape/len(A)    

In [29]:
mape_numba(A_typed, F_typed)

13.325265503992638

In [30]:
%%time
mape_numba(A_typed, F_typed)

Wall time: 16 ms


13.325265503992638

In [23]:
%%time
mape_numba(A_array, F_array)

Wall time: 435 ms


13.325265503992638

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [60]:
ids = reviews["recipe_id"].values
ids

array([ 57993, 142201, 252013, ..., 222001, 354979, 415599], dtype=int64)

In [61]:
def is_pretty(n: int) -> bool:
    last = n%10
    while n//10 > 0:
        n = n//10
        first = n
    return last == first

In [62]:
is_pretty(ids[0]) #57993 - число некрасивое

False

In [63]:
is_pretty(ids[1]) #142201 - число красивое

True

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

In [64]:
%%time
k = 0
for idd in ids:
    if is_pretty(idd):
        k += 1
print(k)

11955
Wall time: 1.68 s


4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [65]:
is_pretty_v = np.vectorize(is_pretty)

In [66]:
%%time
is_pretty_v(ids).sum()

Wall time: 232 ms


11955

4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [90]:
@numba.vectorize()
def is_pretty(n: int) -> bool:
    last = n%10
    while n//10 > 0:
        n = n//10
        first = n
    return last == first

In [91]:
%%time
is_pretty(ids).sum()

Wall time: 216 ms


11955